In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep

import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
import seaborn as sns
import os

from PIL import Image, ImageOps, ImageFilter
import pytesseract
import numpy as np

from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
from typing import Iterable
import concurrent

from tqdm import tqdm


In [17]:
class clasificador(object):
    def __init__(self, clf=None, training=False):
        self.training = training
        self.clf = clf
        if not clf:
            self.training = True

    def transform_image(self, image: Image) -> np.ndarray:
        """aplica transformaciones a las imagenes de cada letra, asume que ya esta en blanco y negro"""
        # transformar a numpy array
        img = np.array(image)

        # convertir en unidimensional
        img = img.flatten()

        # escalar
        img = np.interp(img, (img.min(), img.max()), (-1, +1))

        return img

    def get_images(self, file:Path, n:int=5) -> list:
        # leer la imagen y transformarla a gris
        imagen = Image.open(file)
        imagen = ImageOps.grayscale(imagen)

        # aplicar filtro gaussiano
        imagen = imagen.filter(ImageFilter.GaussianBlur(radius=3))

        # escalarla a la mitad
        imagen = imagen.resize((int(imagen.width/2), int(imagen.height/2)))

        # devolver los datos procesados
        # fila = [imagen, texto]
        # return fila

        # Cortar la imagen en n caracteres:
        # usualmente los captchas tienen un padding (margen interior)
        # al rededor del texto, lo eliminaremos
        padding = 8

        # tambien agregamos un margen al rededor de cada caracter para
        # garantizar que siempre tendremos el caracter dentro del recorte
        margen = 4

        # datos del tamaño de la imagen
        imagen.size
        length = imagen.size[0]
        width = imagen.size[1]
        crop_length = int((length - padding*2)/n)

        imagenes = []

        # recortamos todos los caracteres de la imagen
        for i in range(n):
            imagenes.append(
                self.transform_image(
                    imagen.crop((
                        padding + crop_length*i - margen/2, 0, 
                        padding + crop_length*(i+1)+margen/2, width
                    ))
                )
            )
        return imagenes

    def get_caracteres(self, file: Path) -> list:
        name = file.name
        # tipo = name.split('--')[0]
        texto = name.split('--')[1].split('.')[0]
        return [c for c in texto]

    def procesar(self, file: Path) -> dict:
        """Funcion para procesas un archivo png y romperlo entre el texto que contiene"""
        # obtener los caracteres
        caracteres = self.get_caracteres(file)
        # obtener las imagenes
        imagenes = self.get_images(file, len(caracteres))
        return {'imagenes': imagenes, 'caracteres': caracteres}

    def predict(self, img:Path):
        imagenes = self.get_images(img, 5)
        return self.clf.predict(imagenes)

In [18]:
from joblib import load
from pathlib import Path

clf = load('model.joblib')

def identificar_captcha():
    text = ''.join(clf.predict(Path('captcha.png')))
    return text

In [19]:
url = 'http://localhost:8000/examples/t_api-captcha-conf_via-captchaconfig/'

# cargar el webdriver
driver = webdriver.Chrome()

# cargar la pagina
driver.get(url)

# ids
button_id = 'ValidateCaptchaButton'
input_id = 'CaptchaCode'
captcha_id = 'AppConfigCustomizedCaptcha_CaptchaImage'

sleep(2)
# enviar 10 veces
for i in range(10):
    try:
        WebDriverWait(driver, 10).until(
            EC.all_of(
                EC.element_to_be_clickable((By.ID, button_id)),
                EC.none_of(
                    EC.element_attribute_to_include((By.ID, input_id), 'disabled')
                )
            )
        )
    except:
        print('No se encontro el boton')
        driver.quit()
    finally:
        # obtener la imagen con id captcha_id
        captcha_element = driver.find_element_by_id(captcha_id)
        # obtener el src de la imagen
        captcha_element.screenshot('captcha.png')

        # leer el captcha
        codigo = identificar_captcha()

        # buscar el elemento con id=CaptchaCode
        input_element = driver.find_element(By.ID, input_id)

        # enviar el codigo de captcha
        input_element.send_keys(codigo)
        sleep(5)

        # mostrar por 2 segundos el codigo y luego enviar
        # sleep(2)
        button_element = driver.find_element(By.ID, button_id)
        button_element.click()

driver.quit()



/tmp/ipykernel_20219/2329255769.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  captcha_element = driver.find_element_by_id(captcha_id)
/tmp/ipykernel_20219/2329255769.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  captcha_element = driver.find_element_by_id(captcha_id)
/tmp/ipykernel_20219/2329255769.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  captcha_element = driver.find_element_by_id(captcha_id)
/tmp/ipykernel_20219/2329255769.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  captcha_element = driver.find_element_by_id(captcha_id)
/tmp/ipykernel_20219/2329255769.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  captcha_element = driver.find_element_by_id(captcha_id)


KeyboardInterrupt: 